### **Goals set for the case by myself**:
Create a scalable, easily understandable solution and have some fun (by learning something new).

### **Agenda**
1. My process and choices
2. Assumptions
3. The PokeGraphql class
4. Solving the case
5. Pseudonymization


### **The Request**
The investor requested *name*, *id*, *base_experience*, *weight*, *height*, *types* and *order* of all Pokémon that 
appear in the any of the games red, blue, leafgreen or white.

#### **Follow up questions with investor:**
- When you say "All Pokemon that appear in any of the games, can you specify?"
    - *All Pokemon, that can appear through evolution, gifts, trading, and encounters*
<br>
<br>
- The front_default sprite has gone through several iterations, which do you prefer?
    - *The newest*
<br>
<br>
- In which format do want the data to be delivered?
    - *In a tabular format*
<br>

#### **The request as a Graphql query**
*queries/request.graphql*
```GraphQL
query InvestorRequest {
  pokemon_v2_pokemon(where: {pokemon_v2_pokemongameindices: {pokemon_v2_version: {name: {_in: ['red', 'blue', 'leafgreen', 'white']}}}}) {
    order
    height
    base_experience
    id
    name
    weight
    pokemon_v2_pokemontypes {
      pokemon_v2_type {
        name
      }
    }
  }
}
```

In [ ]:
from pokegraphql import PokeGraphql

In [ ]:
with open("queries/request.graphql") as file:
    query = file.read()

pql = PokeGraphql(query)

pql.df.head()

#### **Format**

In [ ]:
pql.expand_nested()
df = pql.df
df.columns = df.columns.str.replace("pokemon_v2_pokemon_|pokemon_v2_pokemon", "", regex=True)
df.rename({"types_0_pokemon_v2_type.name": "type_0", "types_1_pokemon_v2_type.name":"type_1"}, inplace=True, axis=1)
df["name"] = df["name"].str.capitalize()
df.head(10)

#### **Example: Input Agnostic**

```GraphQL
query Example {
  pokemon_v2_pokemontypepast {
    generation_id
    pokemon_v2_pokemon {
      name
      id
    }
    pokemon_v2_type {
      name
    }
  }
}

```

In [ ]:
with open("queries/pokepasttypes.graphql") as file:
    query = file.read()

diff_input = PokeGraphql(query)
diff_input.expand_nested()
diff_input.df.columns = diff_input.df.columns.str.replace("pokemon_v2_pokemontypepast_", "", regex=True)
diff_input.df.head()

#### **Calculate BMI**

In [ ]:
df["bmi"] = df["weight"] / df["height"]
df.head()

#### **Create front_default sprite url**
The URIs from the graphql beta are invalid, redirecting to the old location of the image files. Luckily the new url structure is based on the the id. So 25.png will get:

![alt text](https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png "Pikachu")

In [ ]:
df["front_default_sprite"] = "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/" + df["id"].astype(str) + ".png"
df.head()

In [ ]:
df.to_parquet("pokemon.parquet", index=False)
#All parquet viewers in vscode are broken...
df.to_csv("pokemon.csv", index=False)

### **Pseudonymisation**
"Begrebet pseudonymisering er i databeskyttelsesforordningen defineret som behandling af personoplysninger på en sådan måde, at personoplysningerne ikke længere kan henføres til en bestemt registreret uden brug af supplerende oplysninger, forudsat at sådanne supplerende oplysninger opbevares separat og er underlagt tekniske og organisatoriske foranstaltninger for at sikre, at personoplysningerne ikke henføres til en identificeret eller identificerbar fysisk person." kilde: https://www.datatilsynet.dk/hvad-siger-reglerne/grundlaeggende-begreber/hvad-er-personoplysninger

In [ ]:
import hashlib
def pseudonymize_name(name, salt):
    encoded_str = f"{salt}{name}".encode("utf8")
    return hashlib.sha256(encoded_str).hexdigest()

pseu_df = df.copy(deep=False)

#Would normally come from a secret manager
salt = "p1k4chu"

identifiers = ["order", "id", "name", "front_default_sprite"]
for id in identifiers: 
    pseu_df[id] = df[id].apply(lambda x: pseudonymize_name(x, salt))

pseu_df.head()

#### **Quasi-identifiers**

In [ ]:
%matplotlib inline
import pandas as pd
res = df["height"].value_counts()
res.plot(kind="hist")

In [ ]:
res = pd.qcut(df["height"], q=4, precision=0, labels=["25%", "50%", "75%", "100%"])
res.value_counts(sort=False).plot(kind="barh")

In [ ]:

pseu_df["weight"] = pd.qcut(df["weight"], q=4, precision=0, labels=["25%", "50%", "75%", "100%"])
pseu_df["height"] = pd.qcut(df["height"], q=4, precision=0, labels=["25%", "50%", "75%", "100%"])
pseu_df["base_experience"] = pd.qcut(df["base_experience"], q=4, precision=0, labels=["25%", "50%", "75%", "100%"])
pseu_df["bmi"] = pd.qcut(df["bmi"], q=4, precision=0, labels=["25%", "50%", "75%", "100%"])

#### **Human friendly pseudonyms for categorical data**

In [ ]:
pseu_df

In [ ]:
import random

pseu_df["types"] = df["type_0"].fillna("") + df["type_1"].fillna("")
res = pseu_df["types"].unique()
random_ids = {}

ids = [number for number in range(0, len(res))]
for types in res:
    id = random.choice(ids)
    random_ids[f"^{types}$"] = str(id)
    ids.remove(id)

for old, new in random_ids.items():
    pseu_df["types"] = pseu_df["types"].str.replace(old, new, regex=True)



In [ ]:
pseu_df

### **Thank you for listening!**